In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import clear_output          
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier

## CALCULO DE HISTORIAL DE CANTIDAD DE POSTULACIONES A LAS DISTINTAS CATEGORIAS PARA UN MISMO USERID

- Leo todas las postulaciones

In [2]:
post_totales = pd.read_csv('post_totales.csv', usecols=['idaviso', 'idpostulante'])
post_totales.drop_duplicates(inplace=True)
post_totales.sort_values(by=['idpostulante'], ascending=True, inplace=True)
post_totales.reset_index(drop=True, inplace=True)
post_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 100.8+ MB


In [3]:
post_totales.head()

,idaviso,idpostulante
0,1112384041,0z5Dmrd
1,1112420060,0z5Dmrd
2,1112428361,0z5JW1r
3,1112309728,0z5JW1r
4,1112292169,0z5JW1r


- Concateno todos los detalles de avisos

In [4]:
avisos_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv', usecols=['idaviso', 'nombre_area'])])
avisos_totales.drop_duplicates(subset=['idaviso'], inplace=True)
avisos_totales.reset_index(drop=True, inplace=True)
avisos_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25288 entries, 0 to 25287
Data columns (total 2 columns):
idaviso        25288 non-null int64
nombre_area    25288 non-null object
dtypes: int64(1), object(1)
memory usage: 395.2+ KB


In [5]:
avisos_totales['nombre_area'].value_counts().size

188

- Label encoding para las 188 areas, y entreno el One Hot Encoding con esos 188 codigos (0-187)

In [6]:
#Fundamental usar para el label encoder una columna DF que contenga TODOS los posibles valores categoricos del nombre de area, es decir, lo saco del DF de TODOS los anuncios.
#Esto es para que si en un set no estan todos, igual los codifique de la misma forma. SI lo entreno con un set que no tiene todos, al ingresar un set con valores nuevos
#va a tirar error porque no tenia esos labels entrenados

#UNA VEZ ENTRENADO EL LABEL ENCODER, Y EL ONEHOTENCODER, USAR SIEMRE EL MISMO PARA EL RESTO DEL NOTEBOOK CODIFICAR LOS SETS DE DATA (TEST Y TRAIN) Y EL SET DE TEST100K

le = preprocessing.LabelEncoder()
avisos_totales['codigo_area'] = le.fit_transform(avisos_totales['nombre_area'].astype(str))

#PARA EL ONE HOT ENCODING, AVISARLE QUE LA CANTIDAD DE VALORES CATEGORICOS ES EL DE TODAS LAS CATEGORIAS EN EL DF DE TODOS LOS ANUNCIOS:188
enc = preprocessing.OneHotEncoder(188)   
enc.fit(avisos_totales['codigo_area'].values.reshape(-1,1))

avisos_totales.head()

,idaviso,nombre_area,codigo_area
0,8725750,Comercial,30
1,17903700,Salud,158
2,1000150677,Transporte,181
3,1000610287,Transporte,181
4,1000872556,Producción,143


- Merge con las postulaciones

In [7]:
post_totales = post_totales.merge(avisos_totales[['idaviso','codigo_area']], how='inner', on='idaviso')
post_totales['codigo_area'] = post_totales['codigo_area'].astype('int16')

post_totales.head()

,idaviso,idpostulante,codigo_area
0,1112384041,0z5Dmrd,154
1,1112384041,0zPOdvr,154
2,1112384041,0zkPkpE,154
3,1112384041,0zr6kGq,154
4,1112384041,126kpz,154


- Armo estructura donde voy a alojar el historial codificado con One Hot Encoding

In [8]:
users_ids_history = post_totales['idpostulante'].to_frame()
users_ids_history.drop_duplicates(inplace=True)
users_ids_history.sort_values(by=['idpostulante'], ascending=True, inplace=True)
users_ids_history.reset_index(drop=True, inplace=True)
users_ids_history.head()

,idpostulante
0,0z5Dmrd
1,0z5JW1r
2,0z5VvGv
3,0zB01pE
4,0zB026d


In [9]:
#Auxiliar
users_ids_history['aux'] = 0

#Encodeo el valor categorico 0 para crear una matriz de encoding vacia
encoded_areas = enc.transform(users_ids_history['aux'].values.reshape(-1,1))
encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

users_ids_history = pd.concat([users_ids_history, encoded_areas_df], axis=1)

del encoded_areas, encoded_areas_df

#Borro las demas columnas para que me queden solo las 188 de encoding. A la primer columna de encoding la vuelvo a setear en 0 ya que al encodear antes el valor
#categorico 0, esa columna esta completa con 1
users_ids_history.drop(columns=['aux'], inplace=True)
users_ids_history[0] = 0

users_ids_history.head()

,idpostulante,0,1,2,3,4,5,6,7,8,...,178,179,180,181,182,183,184,185,186,187
0,0z5Dmrd,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0z5JW1r,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0z5VvGv,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0zB01pE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0zB026d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Organizo la data de la cual voy a sacar el historial, y genero columna de cantidad, para saber cuantas veces postulo a esa categoria

In [10]:
#Quiero ver cuantas veces se postulo a cada categoria, agrupo y saco la cantidad por cada area
post_totales.rename(columns={'idaviso':'count'}, inplace=True)
post_totales = post_totales.groupby(['idpostulante','codigo_area']).count()
post_totales.reset_index(drop=False, inplace=True)
post_totales.sort_values(by='idpostulante', ascending=True, inplace=True)
post_totales.set_index('idpostulante', drop=True, inplace=True)

post_totales.head()

,codigo_area,count
idpostulante,,
0z5Dmrd,154,2
0z5JW1r,5,1
0z5JW1r,30,1
0z5JW1r,51,1
0z5JW1r,82,1


- Extraccion del historial de veces que se postulo a cada categoria

In [11]:
progress = 0
aux = 0
progress_size = (users_ids_history.shape)[0]

while(progress < progress_size):
    
    aux = post_totales.loc[users_ids_history.iat[progress,0],:]

    if(aux.size != 2):
        for index, row in aux.iterrows():
            users_ids_history.iat[progress, np.int(row[0]+1)] = np.int(row[1])
    else:
        users_ids_history.iat[progress, np.int(aux[0]+1)] = np.int(aux[1])  
    
    progress += 1
    clear_output(wait=True)
    print(progress, 'of', progress_size)

346764 of 346764


- Lo exportamos a archivo

In [13]:
users_ids_history.to_csv('historial_cant_post_xuser_xarea.csv', index=False)